# 1. Install and Import Dependencies

In [1]:
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
# !pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

c:\Users\ANAMIKA JHA\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ANAMIKA JHA\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\ANAMIKA JHA\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\ANAMIKA JHA\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# 2. Instantiate Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# 3. Encode and Calculate Sentiment

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [6]:
result = model(tokens)

In [7]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [8]:
int(torch.argmax(result.logits))+1

4

# 4. Collect Reviews

In [9]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [1]:
# link = "https://www.amazon.in/Oppo-10-36-inch-Wi-Fi-Tablet/dp/B0BL75HY31/ref=lp_75024571031_1_1?sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&th=1"
link = "https://www.amazon.in/Handcuffs-Fashion-Waterproof-Athletic-Friendly/dp/B07GTGKW9S/ref=sr_1_1_sspa?keywords=mens+wind+resistant+glove&sr=8-1-spons&sp_csd=d2lkZ2V0TmFtZT1zcF9hdGY&psc=1#customerReviews"
# link = "https://www.amazon.in/Noise-Launched-Bluetooth-Calling-Tracking/product-reviews/B0BJ72WZQ7/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"

r = requests.get(link, headers=HEADERS)
soup = BeautifulSoup(r.content, 'html.parser')
regex = re.compile('review-text-content')

results = soup.find_all(class_= regex) # review-text-content

reviews = [result.get_text(strip=True) for result in results] # review-text-content

NameError: name 'requests' is not defined

In [11]:
results[0].text #open first review

IndexError: list index out of range

In [ ]:
reviews

['It’s good but touch part is not working On th mobile while using it',
 'Product is awesome',
 "Don't expect this will save you during winter...This is my first review for any product but wanted to write it because I felt it when I was coming from Goa to Pune during night ride.Please do not wear during winter... Doesn't help.Finger tips will go numb.",
 'It will easily torn in just 2 -3 months roughly',
 'Avarage',
 'The media could not be loaded.Not that much comfortable while riding, it is okay for normal usage or local data they  commute.Quality is also not that up to mark',
 'Not bad',
 'Gatiya protukt']

# 5. Load Reviews into DataFrame and Score

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def names_purify(names):
    new_list = []

    if names.pop(0) != "Amazon Customer":
        names.pop(0)

    for name in names:
        if name not in new_list:
            new_list.append(name)

    return new_list

In [ ]:
# regex = re.compile('review-text-content')

results = soup.find_all('span', class_= 'a-profile-name')

names = [result.get_text() for result in results]
names = names_purify(names)
names

['kiran',
 'Wangda s.',
 'Prasad Goware',
 'Rajshekar',
 'ashfaq h.',
 'Nishant Kumar Sinha',
 'Abhay Kattewar',
 'Devbrat thakur']

In [ ]:
df = pd.DataFrame({'names': names,'review':reviews})

In [ ]:
df

,names,review
0,kiran,It’s good but touch part is not working On th ...
1,Wangda s.,Product is awesome
2,Prasad Goware,Don't expect this will save you during winter....
3,Rajshekar,It will easily torn in just 2 -3 months roughly
4,ashfaq h.,Avarage
5,Nishant Kumar Sinha,The media could not be loaded.Not that much co...
6,Abhay Kattewar,Not bad
7,Devbrat thakur,Gatiya protukt


In [ ]:
df['review'].iloc[0]

'It’s good but touch part is not working On th mobile while using it'

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
sentiment_score(df['review'].iloc[1])

5

In [ ]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [ ]:
df

,names,review,sentiment
0,kiran,It’s good but touch part is not working On th ...,3
1,Wangda s.,Product is awesome,5
2,Prasad Goware,Don't expect this will save you during winter....,1
3,Rajshekar,It will easily torn in just 2 -3 months roughly,2
4,ashfaq h.,Avarage,1
5,Nishant Kumar Sinha,The media could not be loaded.Not that much co...,3
6,Abhay Kattewar,Not bad,3
7,Devbrat thakur,Gatiya protukt,3


In [ ]:
df['review'].iloc[3]

'It will easily torn in just 2 -3 months roughly'